---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [12]:
import pandas as pd
import numpy as np

def blight_model():
    
    #Loads Data
    train=pd.read_csv('train.csv',encoding = 'ISO-8859-1')
    test=pd.read_csv('test.csv',encoding = 'ISO-8859-1')
    address=pd.read_csv('addresses.csv',encoding = 'ISO-8859-1')
    latlon=pd.read_csv('latlons.csv',encoding = 'ISO-8859-1')
    
    #Retain Required rows of training data
    train=train[(train["compliance"]==0) | (train["compliance"]==1)]
    train=train[~train["hearing_date"].isnull()]
    
    #Join latitude and longitude with every ticket location
    idToLatLon=address.set_index('address').join(latlon.set_index('address'),how='left')
    train=train.set_index('ticket_id').join(idToLatLon.set_index('ticket_id'),how='left')
    test=test.set_index('ticket_id').join(idToLatLon.set_index('ticket_id'),how='left')
    
    #Days b/w issue and hear
    def gap(hear,issue):
        from datetime import datetime
        if(not hear or type(hear)!=str or hear==""):
            return int(np.mean(train["gap_days"]))
        date_format="%Y-%m-%d %H:%M:%S"
        hear_date=datetime.strptime(hear,date_format)
        issue_date=datetime.strptime(issue,date_format)
        difference=hear_date-issue_date
        return difference.days
    
    #Adds gap_days to both train and test
    train["gap_days"]=train.apply(lambda row:gap(str(row["hearing_date"]),str(row["ticket_issued_date"])),axis=1)
    test["gap_days"]=test.apply(lambda row:gap(row["hearing_date"],row["ticket_issued_date"]),axis=1)
    
    #Fill missing
    train.lat.fillna(method='pad', inplace=True)
    train.lon.fillna(method='pad', inplace=True)
    train.state.fillna(method='pad', inplace=True)

    test.lat.fillna(method='pad', inplace=True)
    test.lon.fillna(method='pad', inplace=True)
    test.state.fillna(method='pad', inplace=True)
    
    #Select Required features
    notPresentcols=['payment_amount', 'balance_due','payment_date', 'payment_status', 'collection_status', 'compliance_detail']
    reqSplit= ['agency_name', 'state', 'disposition']
    train_data = pd.get_dummies(train, columns=reqSplit)
    test_data = pd.get_dummies(test, columns=reqSplit)
    remove_total_features=['fine_amount', 'violator_name', 'zip_code', 'country', 'city',
                          'inspector_name', 'violation_street_number', 'violation_street_name',
                          'violation_zip_code', 'violation_description',
                          'mailing_address_str_number', 'mailing_address_str_name',
                          'non_us_str_code',
                          'ticket_issued_date', 'hearing_date', 'grafitti_status', 'violation_code']
    
    remove_from_train=notPresentcols
    train_data.drop(remove_from_train,axis=1,inplace=True)
    train_data.drop(remove_total_features,axis=1,inplace=True)
    test_data.drop(remove_total_features,axis=1,inplace=True)
    
    features_for_training=train_data.columns.drop('compliance')
    features_train=list(features_for_training[:])
    
    for feature in features_for_training:
        if feature not in test_data.columns:
            features_train.remove(feature)
    
    X_train=train_data[features_train]
    y_train=train_data["compliance"]
    X_test=test_data[features_train]
#     y_test=
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.preprocessing import MinMaxScaler
    scale=MinMaxScaler()
    X_train_scaled=scale.fit_transform(X_train)
    X_test_scaled=scale.transform(X_test)
    clf = RandomForestClassifier().fit(X_train_scaled, y_train)
        
    test_index=pd.read_csv('test.csv',encoding = 'ISO-8859-1')
    index=list(test_index["ticket_id"])
    
    y_pred=clf.predict_proba(X_test_scaled)[:,1]
    
    result=pd.Series(y_pred,index=index)
    
    return result# Your answer here
blight_model()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


284932    0.00
285362    0.20
285361    0.10
285338    0.10
285346    0.00
285345    0.00
285347    0.00
285342    1.00
285530    0.00
284989    0.00
285344    0.00
285343    0.00
285340    0.00
285341    0.20
285349    0.10
285348    0.10
284991    0.00
285532    0.10
285406    0.00
285001    0.00
285006    0.00
285405    0.00
285337    0.00
285496    0.00
285497    0.30
285378    0.00
285589    0.00
285585    0.10
285501    0.00
285581    0.00
          ... 
376367    0.00
376366    0.00
376362    0.00
376363    0.20
376365    0.00
376364    0.00
376228    0.05
376265    0.30
376286    0.10
376320    0.00
376314    0.00
376327    0.30
376385    0.10
376435    0.80
376370    0.90
376434    0.00
376459    0.00
376478    0.00
376473    0.10
376484    0.00
376482    0.00
376480    0.00
376479    0.00
376481    0.00
376483    0.00
376496    0.00
376497    0.00
376499    0.00
376500    0.00
369851    0.70
dtype: float64

In [8]:
import pandas as pd
import numpy as np
train=pd.read_csv('train.csv',encoding = 'ISO-8859-1')
test=pd.read_csv('test.csv',encoding = 'ISO-8859-1')
address=pd.read_csv('addresses.csv',encoding = 'ISO-8859-1')
latlons=pd.read_csv('latlons.csv',encoding = 'ISO-8859-1')
print("Train shape: ",train.shape)
print("Test shape: ",test.shape)
print("Address shape: ",address.shape)
print("Latlon shape: ",latlons.shape)

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Train shape:  (250306, 34)
Test shape:  (61001, 27)
Address shape:  (311307, 2)
Latlon shape:  (121769, 3)


In [9]:
train=train[(train["compliance"]==0) | (train["compliance"]==1)]

In [26]:
train[~train["hearing_date"].isnull()].shape,train.shape

((159653, 34), (159880, 34))

In [27]:
train=train[~train["hearing_date"].isnull()]

In [28]:
train.columns,train.shape

(Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
        'violation_street_number', 'violation_street_name',
        'violation_zip_code', 'mailing_address_str_number',
        'mailing_address_str_name', 'city', 'state', 'zip_code',
        'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
        'violation_code', 'violation_description', 'disposition', 'fine_amount',
        'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
        'clean_up_cost', 'judgment_amount', 'payment_amount', 'balance_due',
        'payment_date', 'payment_status', 'collection_status',
        'grafitti_status', 'compliance_detail', 'compliance'],
       dtype='object'), (159653, 34))

In [31]:
test[~test["hearing_date"].isnull()].shape

(58804, 27)

In [32]:
test[~test["ticket_issued_date"].isnull()].shape

(61001, 27)

In [11]:
test.columns

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'grafitti_status'],
      dtype='object')

In [12]:
notPresentcols=['payment_amount', 'balance_due','payment_date', 'payment_status', 'collection_status', 'compliance_detail', 'compliance']

In [ ]:
len(notPresentcols)

In [13]:
latlons.head()

,address,lat,lon
0,"4300 rosa parks blvd, Detroit MI 48208",42.346169,-83.079962
1,"14512 sussex, Detroit MI",42.394657,-83.194265
2,"3456 garland, Detroit MI",42.373779,-82.986228
3,"5787 wayburn, Detroit MI",42.403342,-82.957805
4,"5766 haverhill, Detroit MI",42.407255,-82.946295


In [ ]:
address.set_index('address',inplace=True)

In [ ]:
latlons.set_index('address',inplace=True)

In [ ]:
mixDf=address.join(latlons,how='left')

In [ ]:
mixDf.head()

In [ ]:
train = train.set_index('ticket_id').join(mixDf.set_index('ticket_id'))

In [92]:
def gap(hear,issue):
    from datetime import datetime
    if(not hear or type(hear)!=str or hear==""):
        return int(np.mean(train["gap_days"]))
    date_format="%Y-%m-%d %H:%M:%S"
    hear_date=datetime.strptime(hear,date_format)
    issue_date=datetime.strptime(issue,date_format)
    difference=hear_date-issue_date
#     return [hear,issue,difference]
    return difference.days
def date_time(t):
    from datetime import datetime
    t=str(t)
    date_format="%Y-%m-%d %H:%M:%S"
    date=datetime.strptime(t,date_format)
    return date

In [52]:
gap("2005-03-21 10:30:00","2005-02-22 15:00:00")
date_time("2005-03-21 10:30:00")

datetime.datetime(2005, 3, 21, 10, 30)

In [88]:
int(gap("","2005-03-21 10:30:00"))

72

In [85]:
from datetime import datetime
date_format="%Y-%m-%d %H:%M:%S"
train["gap_days"]=train.apply(lambda row:gap(str(row["hearing_date"]),str(row["ticket_issued_date"])),axis=1)

In [89]:
train["gap_days"].head()

0    369
1    378
5    323
6    253
7    251
Name: gap_days, dtype: int64

In [94]:
test["gap_days"]=test.apply(lambda row:gap(row["hearing_date"],row["ticket_issued_date"]),axis=1)

((2313, 28), (2197, 28))

In [103]:
train.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance,gap_days
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,369
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0,378
5,22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","KASIMU, UKWELI",6478.0,NORTHFIELD,NaN,2755.0,E. 17TH,LOG BEACH,...,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,323
6,18738,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Deerwood Development Group Inc, Deer",8027.0,BRENTWOOD,NaN,476.0,Garfield,Clinton,...,855.0,0.0,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,253
7,18735,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,MT ELLIOTT,NaN,8228.0,Mt. Elliott,Detroit,...,140.0,0.0,140.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,251


In [104]:
feature_to_be_splitted = ['agency_name', 'state', 'disposition']

In [111]:
train_data=train
test_data=test

In [115]:
train_data.columns

Index(['ticket_id', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'zip_code', 'non_us_str_code',
       'country', 'ticket_issued_date', 'hearing_date', 'violation_code',
       'violation_description', 'fine_amount', 'admin_fee', 'state_fee',
       'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount',
       'payment_amount', 'balance_due', 'payment_date', 'payment_status',
       'collection_status', 'grafitti_status', 'compliance_detail',
       'compliance', 'gap_days',
       'agency_name_Buildings, Safety Engineering & Env Department',
       'agency_name_Department of Public Works',
       'agency_name_Detroit Police Department',
       'agency_name_Health Department', 'agency_name_Neighborhood City Halls',
       'state_AK', 'state_AL', 'state_AR', 'state_AZ', 'state_BC', 'state_BL',
       'state_CA', 'state_CO', 's

In [114]:
train_data = pd.get_dummies(train_data, columns=feature_to_be_splitted)
test_data = pd.get_dummies(test_data, columns=feature_to_be_splitted)

ValueError: labels ['agency_name' 'state' 'disposition'] not contained in axis

In [117]:
list_to_remove_train = [
        'balance_due',
        'collection_status',
        'compliance_detail',
        'payment_amount',
        'payment_date',
        'payment_status'
    ]
list_to_remove_all = ['fine_amount', 'violator_name', 'zip_code', 'country', 'city',
                          'inspector_name', 'violation_street_number', 'violation_street_name',
                          'violation_zip_code', 'violation_description',
                          'mailing_address_str_number', 'mailing_address_str_name',
                          'non_us_str_code',
                          'ticket_issued_date', 'hearing_date', 'grafitti_status', 'violation_code']

In [118]:
train_data.drop(list_to_remove_train, axis=1, inplace=True)
train_data.drop(list_to_remove_all, axis=1, inplace=True)
test_data.drop(list_to_remove_all, axis=1, inplace=True)

In [122]:
train_data.columns

Index(['ticket_id', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'compliance', 'gap_days',
       'agency_name_Buildings, Safety Engineering & Env Department',
       'agency_name_Department of Public Works',
       'agency_name_Detroit Police Department',
       'agency_name_Health Department', 'agency_name_Neighborhood City Halls',
       'state_AK', 'state_AL', 'state_AR', 'state_AZ', 'state_BC', 'state_BL',
       'state_CA', 'state_CO', 'state_CT', 'state_DC', 'state_DE', 'state_FL',
       'state_GA', 'state_HI', 'state_IA', 'state_ID', 'state_IL', 'state_IN',
       'state_KS', 'state_KY', 'state_LA', 'state_MA', 'state_MD', 'state_ME',
       'state_MI', 'state_MN', 'state_MO', 'state_MS', 'state_MT', 'state_NB',
       'state_NC', 'state_ND', 'state_NH', 'state_NJ', 'state_NM', 'state_NV',
       'state_NY', 'state_OH', 'state_OK', 'state_ON', 'state_OR', 'state_PA',
       'state_PR', 'state_QC', 'state_QL', 'state_RI', '

In [124]:
train_features = train_data.columns.drop('compliance')

In [148]:
train_features = train_data.columns.drop('compliance')
test_data.columns

Index(['ticket_id', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'gap_days',
       'agency_name_Buildings, Safety Engineering & Env Department',
       'agency_name_Department of Public Works',
       'agency_name_Detroit Police Department', 'state_AK', 'state_AL',
       'state_AR', 'state_AZ', 'state_BC', 'state_CA', 'state_CO', 'state_CT',
       'state_DC', 'state_DE', 'state_FL', 'state_GA', 'state_HI', 'state_IA',
       'state_ID', 'state_IL', 'state_IN', 'state_KS', 'state_KY', 'state_LA',
       'state_MA', 'state_MD', 'state_ME', 'state_MI', 'state_MN', 'state_MO',
       'state_MS', 'state_MT', 'state_NB', 'state_NC', 'state_ND', 'state_NH',
       'state_NJ', 'state_NM', 'state_NV', 'state_NY', 'state_OH', 'state_OK',
       'state_ON', 'state_OR', 'state_PA', 'state_PR', 'state_QC', 'state_QL',
       'state_RI', 'state_SC', 'state_SD', 'state_TN', 'state_TX', 'state_UK',
       'state_UT', 'state_VA', 'state_VI', 'st

In [149]:
train_features_set = set(train_features)
len(train_features_set)

76

In [150]:
for feature in set(train_features):
        if feature not in test_data:
            train_features_set.remove(feature)
            print(feature)
# train_features = list(train_features_set)

agency_name_Neighborhood City Halls
agency_name_Health Department
state_BL
